In [ ]:
# https://medium.com/grabngoinfo/recommendation-system-item-based-collaborative-filtering-f5078504996a
# https://towardsdatascience.com/building-and-testing-recommender-systems-with-surprise-step-by-step-d4ba702ef80b
# https://www.youtube.com/watch?v=fEd1p8-3S7w&ab_channel=JonCavallieMester

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# 1. Load data

rating df

In [ ]:
df_rating = pd.read_csv('ratings.csv')

In [ ]:
df_rating.shape

In [ ]:
df_rating.head()

movie name df

In [ ]:
df_name = pd.read_csv('movies.csv')

In [ ]:
df_name.shape

In [ ]:
df_name.head()

join both

In [ ]:
df = pd.merge(df_rating, df_name, on='movieId', how='inner')

In [ ]:
df = df[['userId','movieId','title','rating']]

In [ ]:
df.shape

In [ ]:
df.head()

# 2. Check data quality

data type

In [ ]:
df.info()

unique value

In [ ]:
df.shape

In [ ]:
for i in df.columns:
    print('Columns name: ', i)
    print('Unique value: ', df[i].unique())
    print('Count unique value: ', df[i].nunique())
    print('-'*10)

missing value

In [ ]:
df.isnull().sum()

blank value

In [ ]:
(df == '').sum()

check movie id

In [ ]:
movieid_counts = df.groupby('title').agg(number_movieid=('movieId', pd.Series.nunique)).reset_index()

In [ ]:
multiple_movieid = movieid_counts[movieid_counts['number_movieid'] > 1]

In [ ]:
multiple_movieid

In [ ]:
max_movieid_df = df.groupby('title')['movieId'].max().reset_index()

In [ ]:
df = pd.merge(df, max_movieid_df, on='title', how='inner')

In [ ]:
df = df.rename(columns={'movieId_y': 'movieId'})

In [ ]:
df = df[['userId','movieId','title','rating']]

In [ ]:
df

check title

In [ ]:
title_counts = df.groupby('movieId').agg(number_title=('title', pd.Series.nunique)).reset_index()

In [ ]:
multiple_title = title_counts[title_counts['number_title'] > 1]

In [ ]:
multiple_title

unique value (again)

In [ ]:
df.shape

In [ ]:
for i in df.columns:
    print('Columns name: ', i)
    print('Unique value: ', df[i].unique())
    print('Count unique value: ', df[i].nunique())
    print('-'*10)

In [ ]:
df

# 3. Exploratory data analysis (EDA)

columns name

In [ ]:
df.columns

rating max/min/distribution

In [ ]:
df['rating'].max()

In [ ]:
df['rating'].min()

In [ ]:
plt.figure(figsize=(8, 6))
plt.hist(df['rating'], bins=10, color='skyblue', edgecolor='black')
plt.xlabel('Rating')
plt.ylabel('Frequency')
plt.title('Distribution of Ratings')
plt.grid(True)
plt.show()

average rating by title

In [ ]:
avg_rating_by_title = df.groupby('title')['rating'].mean()

In [ ]:
avg_rating_by_title_sorted = avg_rating_by_title.sort_values(ascending=False)

In [ ]:
avg_rating_by_title_sorted

average rating by userId

In [ ]:
avg_rating_by_userId = df.groupby('userId')['rating'].mean()

In [ ]:
avg_rating_by_userId_sorted = avg_rating_by_userId.sort_values(ascending=False)

In [ ]:
avg_rating_by_userId_sorted

count userId by title

In [ ]:
count_userId_by_title = df.groupby('title')['userId'].nunique()

In [ ]:
count_userId_by_title_sorted = count_userId_by_title.sort_values(ascending=False)

In [ ]:
count_userId_by_title_sorted

count title by userId

In [ ]:
count_title_by_userId = df.groupby('userId')['title'].nunique()

In [ ]:
count_title_by_userId_sorted = count_title_by_userId.sort_values(ascending=False)

In [ ]:
count_title_by_userId_sorted

# 4. Model - user based

pivot

In [ ]:
user_item_metrix = df.pivot_table(index='userId', columns='title', values='rating')

In [ ]:
user_item_metrix

mean normalize

In [ ]:
# subtract by user's mean like KNNWithMean in surprise
# Since some people tend to give a higher rating than others, we normalize the rating by extracting the average rating of each user.

In [ ]:
avg_rating_by_userId_sorted

In [ ]:
user_item_metrix_norm = user_item_metrix.subtract(avg_rating_by_userId_sorted, axis = 'rows')

In [ ]:
user_item_metrix_norm

In [ ]:
user_item_metrix_norm_filled = user_item_metrix_norm.fillna(0)

In [ ]:
user_item_metrix_norm_filled

similarity

In [ ]:
user_similarity = cosine_similarity(user_item_metrix_norm_filled)

In [ ]:
user_item_metrix_norm_filled.index

In [ ]:
user_similarity

In [ ]:
user_similarity = pd.DataFrame(user_similarity, index=user_item_metrix_norm_filled.index, columns=user_item_metrix_norm_filled.index)

In [ ]:
user_similarity

# 5. Prediction - user based

setup

In [ ]:
picked_userid = 407

In [ ]:
# set top similar user (k) = 10
k = 10

top similar user

In [ ]:
similar_users = user_similarity[picked_userid].sort_values(ascending=False)[:k]

In [ ]:
similar_users

unwatched movie

In [ ]:
picked_userid_unwatched = user_item_metrix_norm.T[407][user_item_metrix_norm.T[picked_userid].isna()]

In [ ]:
picked_userid_unwatched

In [ ]:
user_item_metrix_norm

score

In [ ]:
similar_users.index.tolist()

In [ ]:
len(picked_userid_unwatched.index.tolist())

In [ ]:
similar_user_movies = user_item_metrix_norm.loc[similar_users.index.tolist()][picked_userid_unwatched.index.tolist()]

In [ ]:
similar_user_movies

In [ ]:
item_list = []
item_score_list = []
item_score_predicted_list = []

In [ ]:
for i in similar_user_movies.columns:
    
    sum_value_weight = 0
    sum_weight = 0
    movie_rating = similar_user_movies[i]
    for u in similar_users.index:
        if pd.isna(movie_rating[u]) == False:
            score = similar_users[u] * movie_rating[u]
            sum_value_weight += score
            sum_weight += similar_users[u]

    if sum_weight == 0:
        item_score = 0
    else:
        item_score = sum_value_weight/sum_weight

    item_list.append(i)
    item_score_list.append(item_score)
    item_score_predicted_list.append(item_score + avg_rating_by_userId_sorted[picked_userid])

In [ ]:
df_predict = pd.DataFrame({
    'title': item_list,
    'ranking': item_score_list,
    'ranking_predicted':item_score_predicted_list
})

In [ ]:
df_predict.sort_values(by='ranking', ascending=False).iloc[:5]

In [ ]:
df_predict.sort_values(by='ranking_predicted', ascending=False).iloc[:5]

In [ ]:
****

# 6. Model - item based

pivot

In [ ]:
item_user_metrix = df.pivot_table(index='title', columns='userId', values='rating')

In [ ]:
item_user_metrix

mean normalize

In [ ]:
# subtract by movie's mean like KNNWithMean in surprise

In [ ]:
avg_rating_by_title_sorted

In [ ]:
item_user_metrix_norm = item_user_metrix.subtract(avg_rating_by_title_sorted, axis = 'rows')

In [ ]:
item_user_metrix_norm

In [ ]:
item_user_metrix_norm_filled = item_user_metrix_norm.fillna(0)

similarity

In [ ]:
item_user_metrix_norm_filled

In [ ]:
item_similarity = cosine_similarity(item_user_metrix_norm_filled)

In [ ]:
item_user_metrix_norm_filled.index

In [ ]:
item_similarity

In [ ]:
item_similarity = pd.DataFrame(item_similarity, index=item_user_metrix_norm_filled.index, columns=item_user_metrix_norm_filled.index)

In [ ]:
item_similarity

In [ ]:
# diagonal is zero due to all zero in the movie

# 7. Prediction - user based

setup

In [ ]:
picked_userid = 407

In [ ]:
k = 10

top similar user

In [ ]:
similar_users = user_similarity[picked_userid].sort_values(ascending=False)[:k]

In [ ]:
similar_users

watched movie

In [ ]:
item_user_metrix_norm

In [ ]:
***

In [ ]:
picked_title_watched = user_item_metrix_norm.T[407][user_item_metrix_norm.T[picked_userid].isna()]

In [ ]:
picked_userid_unwatched

In [ ]:
user_item_metrix_norm